In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
import io
from google.colab import drive
import threading
import multiprocessing
import time
import tensorflow as tf
drive.mount('/content/drive')
import tensorflow as tf
import os.path
tf.test.gpu_device_name()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/device:GPU:0'

In [ ]:
import os.path

train_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sales_train_validation.csv")
calendar_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar.csv")
sellprices2_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices2.csv")
calendar_data2 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar2.csv")

days_num = 1913

cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "wm_yr_wk", "wday", "month", "year", "d",
        "Religious", "National", "Cultural", "Sporting", "snap"] #"price", "price_down", "price_up", "value"は後程追加
train_data2a0 = pd.DataFrame(index=[ij for ij in range(days_num)], columns=cols)

start_n = 0

thread_num = 4 #スレッド数
process_num = 8 #プロセス数

print(len(train_data))

current_num = [-1 for jj in range(thread_num + process_num)]  # 実行中のファイル番号(重複処理を防ぐ)

def stap(ST):
        return calendar_data2["snap_" + ST]

def traindata2_making(th_num):
    for ii in range(start_n, len(train_data)):
        if ((ii in current_num) == False): #その行は処理されていない
            current_num[th_num] = ii
            if (os.path.isfile("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv") == False):  # ファイルがまだ作成されていない
                train_data2a = train_data2a0.copy()
                tra_sir = train_data.iloc[ii]
                for jj in range(days_num):train_data2a.iloc[jj,0:6] = tra_sir[0:6]
                train_data2a.iloc[:,6] = calendar_data.iloc[:,1] #週
                train_data2a.iloc[:,7:11] = calendar_data.iloc[:,3:7] #年, 月, 曜日
                train_data2a.iloc[:,11:15] = calendar_data2.iloc[:,1:5] #イベント情報
                train_data2a.iloc[:,15] = stap(tra_sir[5]) #snap

                sap_mindata = sellprices2_data[sellprices2_data.item_id == tra_sir[1]]  # sellpricesの中から必要なデータを抽出
                sap_mindata = sap_mindata[sap_mindata.store_id == tra_sir[4]]
                sap_mindata = sap_mindata.drop(["store_id", "item_id"], axis=1)

                train_data2a = pd.merge(train_data2a, sap_mindata, on='wm_yr_wk', how='left') #価格とその変化

                train_data2a["value"] = tra_sir[6:].to_list()

                train_data2a.to_csv(
                    "/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv", index=False)
            current_num[th_num] = -1

thread_list = []
with tf.device('/cpu:0'):
  for i in range(thread_num):
      thread = threading.Thread(target=traindata2_making, args=(i,))
      thread.start()
      thread_list.append(thread)
      print(i)
      sleep_seconds = 1
      time.sleep(sleep_seconds)

with tf.device('/gpu:0'):
  for i2 in range(thread_num, thread_num + process_num):
      mp = threading.Thread(target=traindata2_making, args=(i2,))
      mp.start()
      thread_list.append(thread)
      print(i2)
      sleep_seconds = 1
      time.sleep(sleep_seconds)

while(True):
  print('\r', end='')
  print(current_num, end='')
  sleep_seconds = 2
  time.sleep(sleep_seconds)

for thread in thread_list:
    thread.join()

30490
0
1
2
3


KeyboardInterrupt: ignored

In [ ]:
ii = 1413
if (os.path.isfile("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv") == False):  # ファイルがまだ作成されていない
  train_data2a = train_data2a0.copy()
  tra_sir = train_data.iloc[ii]
  for jj in range(days_num):train_data2a.iloc[jj,0:6] = tra_sir[0:6]
  train_data2a.iloc[:,6] = calendar_data.iloc[:,1] #週
  train_data2a.iloc[:,7:11] = calendar_data.iloc[:,3:7] #年, 月, 曜日
  train_data2a.iloc[:,11:15] = calendar_data2.iloc[:,1:5] #イベント情報
  train_data2a.iloc[:,15] = stap(tra_sir[5]) #snap

  sap_mindata = sellprices2_data[sellprices2_data.item_id == tra_sir[1]]  # sellpricesの中から必要なデータを抽出
  sap_mindata = sap_mindata[sap_mindata.store_id == tra_sir[4]]
  sap_mindata = sap_mindata.drop(["store_id", "item_id"], axis=1)

  train_data2a = pd.merge(train_data2a, sap_mindata, on='wm_yr_wk', how='left') #価格とその変化

  train_data2a["value"] = tra_sir[6:].to_list()

  train_data2a.to_csv(
      "/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv", index=False)
else: print("files exist")

files exist


In [ ]:
df = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(1413) + ".csv")

In [ ]:
os.remove("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(1413) + ".csv")

In [ ]:
main_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_0.csv")
main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari.csv", index=False)
with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu.txt", mode='w') as f:
    f.write(str(1))

In [ ]:
main_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari.csv")
with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu.txt") as f:
    ii = int(f.read())
max_ii = 30490
error_flag = False

while True:
  try:
    df = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv")
  except:
    print(str(ii)+ "doesn't exist")
    error_flag = True
  else:
    main_data = pd.concat([main_data, df])
    print('\r', end='')
    print(ii, end='')
    if (ii%100 == 0):
      if (error_flag == True): sys.exit(1)
      with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu.txt", mode='w') as f:
        f.write(str(ii))
      main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari.csv", index=False)
  finally:
    ii += 1
    
  

main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_all.csv", index=False)

1400

In [ ]:
with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu.txt") as f:
    ii = int(f.read()) + 1
main_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv")
main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari2.csv", index=False)
with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu2.txt", mode='w') as f:
    f.write(str(ii))

In [ ]:
main_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari2.csv")
with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu2.txt") as f:
    ii = int(f.read())
max_ii = 30490
error_flag = False

while True:
  try:
    df = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv")
  except:
    print(str(ii)+ "doesn't exist")
    error_flag = True
  else:
    main_data = pd.concat([main_data, df])
    print('\r', end='')
    print(ii, end='')
    if (ii%100 == 0):
      if (error_flag == True): sys.exit(1)
      with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu2.txt", mode='w') as f:
        f.write(str(ii))
      main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari2.csv", index=False)
  finally:
    ii += 1
    
  

main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_all.csv", index=False)

22486

In [ ]:
with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu2.txt") as f:
    ii = int(f.read()) + 1
main_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv")
main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari3.csv", index=False)
with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu3.txt", mode='w') as f:
    f.write(str(ii))

In [ ]:
main_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari3.csv")
with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu3.txt") as f:
    ii = int(f.read())
max_ii = 30490
error_flag = False

while True:
  try:
    df = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv")
  except:
    print(str(ii)+ "doesn't exist")
    error_flag = True
  else:
    main_data = pd.concat([main_data, df])
    print('\r', end='')
    print(ii, end='')
    if (ii%100 == 0):
      if (error_flag == True): sys.exit(1)
      with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu3.txt", mode='w') as f:
        f.write(str(ii))
      main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari3.csv", index=False)
  finally:
    ii += 1
    
  

main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari3.csv", index=False)

In [ ]:
mda1 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari.csv")
mda2 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari2.csv")
mda3 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari3.csv")

main_data = pd.concat([mda1, mda2, mda3])
main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train2a_all.csv", index=False)

In [ ]:
size_cols = {"wday": 'int8', "month": 'int8', "year": 'int8',"Religious": 'int8', "National": 'int8',
             "Cultural": 'int8', "Sporting": 'int8', "snap": 'int8'}

m1 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train2a_all.csv", dtype=size_cols)
m1 = m1.drop_duplicates(subset='id')
m1_id = m1['id'].values.tolist()
m2 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sales_train_validation.csv")
m2_id = m2['id'].values.tolist()

print(len(m1_id))

for ii in range(len(m1_id)):
  m2_id.remove(m1_id[ii])
  print("\r"+ str(ii), end=" ")

m2_export = pd.DataFrame({'id': m2_id})

m2_export.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train2a_add_row.csv", index=False)



30490
30489 

In [ ]:
import os.path

train_data0 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train2a_add_row.csv")
train_data1 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sales_train_validation.csv")
train_data = pd.merge(train_data0, train_data1, on='id')


calendar_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar.csv")
sellprices2_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices2.csv")
calendar_data2 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar2.csv")

days_num = 1913

cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "wm_yr_wk", "wday", "month", "year", "d",
        "Religious", "National", "Cultural", "Sporting", "snap"] #"price", "price_down", "price_up", "value"は後程追加
train_data2a0 = pd.DataFrame(index=[ij for ij in range(days_num)], columns=cols)

start_n = 0

thread_num = 4 #スレッド数
process_num = 8 #プロセス数

print(len(train_data))

current_num = [-1 for jj in range(thread_num + process_num)]  # 実行中のファイル番号(重複処理を防ぐ)

def stap(ST):
        return calendar_data2["snap_" + ST]

def traindata2_making(th_num):
    for ii in range(len(train_data)):
        if ((ii in current_num) == False): #その行は処理されていない
            current_num[th_num] = ii
            if (os.path.isfile("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_add" + str(ii) + ".csv") == False):  # ファイルがまだ作成されていない
                train_data2a = train_data2a0.copy()
                tra_sir = train_data.iloc[ii]
                for jj in range(days_num):train_data2a.iloc[jj,0:6] = tra_sir[0:6]
                train_data2a.iloc[:,6] = calendar_data.iloc[:,1] #週
                train_data2a.iloc[:,7:11] = calendar_data.iloc[:,3:7] #年, 月, 曜日
                train_data2a.iloc[:,11:15] = calendar_data2.iloc[:,1:5] #イベント情報
                train_data2a.iloc[:,15] = stap(tra_sir[5]) #snap

                sap_mindata = sellprices2_data[sellprices2_data.item_id == tra_sir[1]]  # sellpricesの中から必要なデータを抽出
                sap_mindata = sap_mindata[sap_mindata.store_id == tra_sir[4]]
                sap_mindata = sap_mindata.drop(["store_id", "item_id"], axis=1)

                train_data2a = pd.merge(train_data2a, sap_mindata, on='wm_yr_wk', how='left') #価格とその変化

                train_data2a["value"] = tra_sir[6:].to_list()

                train_data2a.to_csv(
                    "/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_add" + str(ii) + ".csv", index=False)
            current_num[th_num] = -1

thread_list = []
with tf.device('/cpu:0'):
  for i in range(thread_num):
      thread = threading.Thread(target=traindata2_making, args=(i,))
      thread.start()
      thread_list.append(thread)
      print(i)
      sleep_seconds = 1
      time.sleep(sleep_seconds)

with tf.device('/gpu:0'):
  for i2 in range(thread_num, thread_num + process_num):
      mp = threading.Thread(target=traindata2_making, args=(i2,))
      mp.start()
      thread_list.append(thread)
      print(i2)
      sleep_seconds = 1
      time.sleep(sleep_seconds)

while(True):
  print('\r', end='')
  print(current_num, end='')
  sleep_seconds = 30
  time.sleep(sleep_seconds)

for thread in thread_list:
    thread.join()

In [ ]:
import os.path

train_data0 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train2a_add_row.csv")
train_data1 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sales_train_validation.csv")
train_data = pd.merge(train_data0, train_data1, on='id')


calendar_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar.csv")
sellprices2_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices2.csv")
calendar_data2 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar2.csv")

days_num = 1913

cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "wm_yr_wk", "wday", "month", "year", "d",
        "Religious", "National", "Cultural", "Sporting", "snap"] #"price", "price_down", "price_up", "value"は後程追加
train_data2a0 = pd.DataFrame(index=[ij for ij in range(days_num)], columns=cols)

for ii in range(7182):
  if (os.path.isfile("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_add" + str(ii) + ".csv") == False):  # ファイルがまだ作成されていない
    train_data2a = train_data2a0.copy()
    tra_sir = train_data.iloc[ii]
    for jj in range(days_num):train_data2a.iloc[jj,0:6] = tra_sir[0:6]
    train_data2a.iloc[:,6] = calendar_data.iloc[:,1] #週
    train_data2a.iloc[:,7:11] = calendar_data.iloc[:,3:7] #年, 月, 曜日
    train_data2a.iloc[:,11:15] = calendar_data2.iloc[:,1:5] #イベント情報
    train_data2a.iloc[:,15] = stap(tra_sir[5]) #snap

    sap_mindata = sellprices2_data[sellprices2_data.item_id == tra_sir[1]]  # sellpricesの中から必要なデータを抽出
    sap_mindata = sap_mindata[sap_mindata.store_id == tra_sir[4]]
    sap_mindata = sap_mindata.drop(["store_id", "item_id"], axis=1)

    train_data2a = pd.merge(train_data2a, sap_mindata, on='wm_yr_wk', how='left') #価格とその変化

    train_data2a["value"] = tra_sir[6:].to_list()

    train_data2a.to_csv(
        "/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_add" + str(ii) + ".csv", index=False)

In [ ]:
main_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_add" + str(0) + ".csv")
main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari_add.csv", index=False)
with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu_add.txt", mode='w') as f:
    f.write(str(1))

In [ ]:
size_cols = {"wday": 'int8', "month": 'int8', "year": 'int8',"Religious": 'int8', "National": 'int8',
             "Cultural": 'int8', "Sporting": 'int8', "snap": 'int8'}

main_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari_add.csv", dtype=size_cols)
with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu_add.txt") as f:
    ii = int(f.read())
max_ii = 7181
error_flag = False

while True:
  try:
    df = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_add" + str(ii) + ".csv", dtype=size_cols)
  except:
    print(str(ii)+ "doesn't exist")
    error_flag = True
  else:
    main_data = pd.concat([main_data, df])
    print('\r', end='')
    print(ii, end='')
    if (ii%100 == 0):
      if (error_flag == True): sys.exit(1)
      with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_syukeityu_add.txt", mode='w') as f:
        f.write(str(ii))
      main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari_add.csv", index=False)
  finally:
    ii += 1
    if (ii > max_ii): break
    
  

main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari_add.csv", index=False)

71817182doesn't exist


In [ ]:
import gc

size_cols = {"wday": 'int8', "month": 'int8', "year": 'int8',"Religious": 'int8', "National": 'int8',
             "Cultural": 'int8', "Sporting": 'int8', "snap": 'int8'}
mda1 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train2a_all.csv", dtype=size_cols)
mda2 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_kari_add.csv", dtype=size_cols)

main_data = pd.concat([mda1, mda2])
main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train2a_all2.csv", index=False)

del mda1, mda2, main_data
gc.collect()

1462

train2a_all2.csvが正常であるか調べ, 問題がなければtrain2a_all.csvに名称変更

In [ ]:
import gc

size_cols = {"wday": 'int8', "month": 'int8', "year": 'int8',"Religious": 'int8', "National": 'int8',
             "Cultural": 'int8', "Sporting": 'int8', "snap": 'int8'}

mda = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train2a_all.csv", dtype=size_cols)
mda = mda.drop_duplicates()
mda.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train2a_all.csv", index=False)

del mda
gc.collect()


0

d_の形式は扱いづらいので数値に変換

In [ ]:
import gc

size_cols = {"wday": 'int8', "month": 'int8', "year": 'int8',"Religious": 'int8', "National": 'int8',
             "Cultural": 'int8', "Sporting": 'int8', "snap": 'int8'}

mda = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train2a_all.csv", dtype=size_cols)
def d_del(x):
  if ("d_" in x): return x[2:]
  else: return x

mda = mda.drop_duplicates()
mda['d'] = mda['d'].map(d_del)
mda.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train2a_all2.csv", index=False)

del mda
gc.collect()

0

sell_priceのNaNを0に変換

In [ ]:
import gc

size_cols = {"wday": 'int8', "month": 'int8', "year": 'int8',"Religious": 'int8', "National": 'int8',
             "Cultural": 'int8', "Sporting": 'int8', "snap": 'int8'}

mda = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train2a_all2.csv", dtype=size_cols)

mda = mda.fillna({'sell_price': '0'})

mda.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train2a_all2.csv", index=False)

del mda
gc.collect()

330

In [2]:
import gc
mda = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train2a_all2.csv")

x1_data_CA = mda[mda['state_id']=="CA"]
x1_data_TX = mda[mda['state_id']=="TX"]
x1_data_WI = mda[mda['state_id']=="WI"]

x1_data_CA.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x1_data_CA.csv", index=False)
x1_data_TX.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x1_data_TX.csv", index=False)
x1_data_WI.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x1_data_WI.csv", index=False)

del x1_data_CA, x1_data_TX, x1_data_WI
gc.collect()

x2_data_CA_1 = mda[mda['store_id']=="CA_1"]
x2_data_CA_2 = mda[mda['store_id']=="CA_2"]
x2_data_CA_3 = mda[mda['store_id']=="CA_3"]
x2_data_TX_1 = mda[mda['store_id']=="TX_1"]
x2_data_TX_2 = mda[mda['store_id']=="TX_2"]
x2_data_TX_3 = mda[mda['store_id']=="TX_3"]
x2_data_WI_1 = mda[mda['store_id']=="WI_1"]
x2_data_WI_2 = mda[mda['store_id']=="WI_2"]
x2_data_WI_3 = mda[mda['store_id']=="WI_3"]

x2_data_CA_1.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x2_data_CA_1.csv", index=False)
x2_data_CA_2.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x2_data_CA_2.csv", index=False)
x2_data_CA_3.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x2_data_CA_3.csv", index=False)
x2_data_TX_1.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x2_data_TX_1.csv", index=False)
x2_data_TX_2.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x2_data_TX_2.csv", index=False)
x2_data_TX_3.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x2_data_TX_3.csv", index=False)
x2_data_WI_1.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x2_data_WI_1.csv", index=False)
x2_data_WI_2.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x2_data_WI_2.csv", index=False)
x2_data_WI_3.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x2_data_WI_3.csv", index=False)

del x2_data_CA_1 ,x2_data_CA_2 ,x2_data_CA_3 ,x2_data_TX_1 ,x2_data_TX_2 ,x2_data_TX_3 ,x2_data_WI_1 ,x2_data_WI_2 ,x2_data_WI_3
gc.collect()

x3_data_FOODS = mda[mda['cat_id']=="FOODS"]
x3_data_HOUSEHOLD = mda[mda['cat_id']=="HOUSEHOLD"]
x3_data_HOBBIES = mda[mda['cat_id']=="HOBBIES"]

x3_data_FOODS.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x3_data_FOODS.csv", index=False)
x3_data_HOUSEHOLD.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x3_data_HOUSEHOLD.csv", index=False)
x3_data_HOBBIES.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x3_data_HOBBIES.csv", index=False)

del x3_data_FOODS ,x3_data_HOUSEHOLD, x3_data_HOBBIES
gc.collect()

x4_data_FOODS_1 = mda[mda['dept_id']=="FOODS_1"]
x4_data_FOODS_2 = mda[mda['dept_id']=="FOODS_2"]
x4_data_FOODS_3 = mda[mda['dept_id']=="FOODS_3"]
x4_data_HOUSEHOLD_1 = mda[mda['dept_id']=="HOUSEHOLD_1"]
x4_data_HOUSEHOLD_2 = mda[mda['dept_id']=="HOUSEHOLD_2"]
x4_data_HOBBIES_1 = mda[mda['dept_id']=="HOBBIES_1"]
x4_data_HOBBIES_2 = mda[mda['dept_id']=="HOBBIES_2"]

x4_data_FOODS_1.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x4_data_FOODS_1.csv", index=False)
x4_data_FOODS_2.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x4_data_FOODS_2.csv", index=False)
x4_data_FOODS_3.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x4_data_FOODS_3.csv", index=False)
x4_data_HOUSEHOLD_1.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x4_data_HOUSEHOLD_1.csv", index=False)
x4_data_HOUSEHOLD_2.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x4_data_HOUSEHOLD_2.csv", index=False)
x4_data_HOBBIES_1.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x4_data_HOBBIES_1.csv", index=False)
x4_data_HOBBIES_2.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/x4_data_HOBBIES_2.csv", index=False)

del x4_data_FOODS_1 ,x4_data_FOODS_2 ,x4_data_FOODS_3, x4_data_HOUSEHOLD_1 ,x4_data_HOUSEHOLD_2 ,x4_data_HOBBIES_1, x4_data_HOBBIES_2
gc.collect()

0

各種データを学習

In [6]:
with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_m_learn.txt", mode='w') as f:
    f.write(str(0))

In [9]:
import pickle
import lightgbm as lgb
import gc
from sklearn.model_selection import train_test_split

with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_m_learn.txt") as f:
    ii_start = int(f.read())

x_list_0 = ["x2_data_CA_1","x2_data_CA_2","x2_data_CA_3","x2_data_TX_1","x2_data_TX_2","x2_data_TX_3","x2_data_WI_1","x2_data_WI_2","x2_data_WI_3","x3_data_FOODS","x3_data_HOUSEHOLD","x3_data_HOBBIES","x4_data_FOODS_1","x4_data_FOODS_2","x4_data_FOODS_3","x4_data_HOUSEHOLD_1","x4_data_HOUSEHOLD_2","x4_data_HOBBIES_1","x4_data_HOBBIES_2"]
x_list_1 = ["x1_data_CA", "x1_data_TX", "x1_data_WI", "x3_data_FOODS", "x3_data_HOUSEHOLD", "x3_data_HOBBIES"]
x_list = x_list_0 + x_list_1
in_cols = ["wday","month","year","d","Religious","National","Cultural","Sporting","snap","price_down","price_up","sell_price"]
cat_list = ["wday","month","year","d","Religious","National","Cultural","Sporting","snap","price_down","price_up"]
size_cols = {"wday": 'int8', "month": 'int8', "year": 'int8',"Religious": 'int8', "National": 'int8',
             "Cultural": 'int8', "Sporting": 'int8', "snap": 'int8'}

for ij in range(ii_start, len(x_list)):
  tt = x_list[ij]
  print(tt + ": train start")

  mda = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/" + tt + ".csv", dtype=size_cols)

  for count in range(3):

    train_set, test_set = train_test_split(mda, test_size=0.2, random_state=4)

    train_input = train_set[in_cols]
    for rr in cat_list: train_input[rr] = train_input[rr].astype("category")
    train_result = train_set["value"]
    test_input = test_set[in_cols]
    for rr in cat_list: test_input[rr] = test_input[rr].astype("category")
    test_result = test_set["value"]

    lgb_train = lgb.Dataset(train_input, train_result)
    lgb_test = lgb.Dataset(test_input, test_result)
    # 学習用パラメータ
    lgbm_params = {
      'objective': 'regression',
      'metric': 'rmse',
      "verbosity": 0,}

    clf = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_test)

    print("train end")

    file = '/content/drive/My Drive/google_notebook/M5_traindata2/trained_model_' + tt + '_'+ str(count) +'.pkl'
    pickle.dump(clf, open(file, 'wb'))

    del clf, train_input
    gc.collect()

  del mda
  gc.collect()

  with open("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_m_learn.txt", mode='w') as f:
    f.write(str(ij+1))

x2_data_CA_1: train start


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


ストリーミング出力は最後の 5000 行に切り捨てられました。
[68]	valid_0's rmse: 4.06568
[69]	valid_0's rmse: 4.06568
[70]	valid_0's rmse: 4.06568
[71]	valid_0's rmse: 4.06568
[72]	valid_0's rmse: 4.06568
[73]	valid_0's rmse: 4.06568
[74]	valid_0's rmse: 4.06568
[75]	valid_0's rmse: 4.06568
[76]	valid_0's rmse: 4.06568
[77]	valid_0's rmse: 4.06568
[78]	valid_0's rmse: 4.06568
[79]	valid_0's rmse: 4.06568
[80]	valid_0's rmse: 4.06568
[81]	valid_0's rmse: 4.06568
[82]	valid_0's rmse: 4.06568
[83]	valid_0's rmse: 4.06568
[84]	valid_0's rmse: 4.06568
[85]	valid_0's rmse: 4.06568
[86]	valid_0's rmse: 4.06568
[87]	valid_0's rmse: 4.06568
[88]	valid_0's rmse: 4.06568
[89]	valid_0's rmse: 4.06568
[90]	valid_0's rmse: 4.06568
[91]	valid_0's rmse: 4.06568
[92]	valid_0's rmse: 4.06568
[93]	valid_0's rmse: 4.06568
[94]	valid_0's rmse: 4.06568
[95]	valid_0's rmse: 4.06568
[96]	valid_0's rmse: 4.06568
[97]	valid_0's rmse: 4.06568
[98]	valid_0's rmse: 4.06568
[99]	valid_0's rmse: 4.06568
[100]	valid_0's rmse: 4.06568
train end


提出用データの入力データを作成

In [ ]:
import os.path

days_start = 1942
days_end = 1969

train_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sales_train_validation.csv")
calendar_data_0 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar.csv")
calendar_data = calendar_data_0[days_start-1:].reset_index(drop=True)
sellprices2_data_0 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices2.csv")
sellprices2_data = sellprices2_data_0[days_start-1:].reset_index(drop=True)
calendar_data2_0 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar2.csv")
calendar_data2 = calendar_data2_0[days_start-1:].reset_index(drop=True)

days_num = days_end - days_start + 1

days_list = ["d_" + str(ii) for ii in range(days_start, days_end + 1)]

cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "wm_yr_wk", "wday", "month", "year", "d",
        "Religious", "National", "Cultural", "Sporting", "snap"] #"price", "price_down", "price_up", "value"は後程追加
train_data2a0 = pd.DataFrame(index=[ij for ij in range(days_num)], columns=cols)

start_n = 0

thread_num = 4 #スレッド数
process_num = 8 #プロセス数

print(len(train_data))

current_num = [-1 for jj in range(thread_num + process_num)]  # 実行中のファイル番号(重複処理を防ぐ)

def stap(ST):
        return calendar_data2["snap_" + ST]

def traindata2_making(th_num):
    for ii in range(start_n, len(train_data)):
        if ((ii in current_num) == False): #その行は処理されていない
            current_num[th_num] = ii
            if (os.path.isfile("/content/drive/My Drive/google_notebook/M5_traindata2/export_data2a/export2a_" + str(ii) + ".csv") == False):  # ファイルがまだ作成されていない
                train_data2a = train_data2a0.copy()
                tra_sir = train_data.iloc[ii]
                for jj in range(days_num):train_data2a.iloc[jj,0:6] = tra_sir[0:6]
                train_data2a.iloc[:,6] = calendar_data.iloc[: ,1] #週
                train_data2a.iloc[:,7:11] = calendar_data.iloc[: ,3:7] #年, 月, 曜日
                train_data2a.iloc[:,11:15] = calendar_data2.iloc[: ,1:5] #イベント情報
                tds_kari = stap(tra_sir[5]) #snap
                train_data2a.iloc[:,15] = tds_kari

                sap_mindata = sellprices2_data[sellprices2_data.item_id == tra_sir[1]]  # sellpricesの中から必要なデータを抽出
                sap_mindata = sap_mindata[sap_mindata.store_id == tra_sir[4]]
                sap_mindata = sap_mindata.drop(["store_id", "item_id"], axis=1)

                train_data2a = pd.merge(train_data2a, sap_mindata, on='wm_yr_wk', how='left') #価格とその変化

                train_data2a.to_csv(
                    "/content/drive/My Drive/google_notebook/M5_traindata2/export_data2a/export2a_" + str(ii) + ".csv", index=False)
            current_num[th_num] = -1

thread_list = []
with tf.device('/cpu:0'):
  for i in range(thread_num):
      thread = threading.Thread(target=traindata2_making, args=(i,))
      thread.start()
      thread_list.append(thread)
      print(i)
      sleep_seconds = 1
      time.sleep(sleep_seconds)

with tf.device('/gpu:0'):
  for i2 in range(thread_num, thread_num + process_num):
      mp = threading.Thread(target=traindata2_making, args=(i2,))
      mp.start()
      thread_list.append(thread)
      print(i2)
      sleep_seconds = 1
      time.sleep(sleep_seconds)

while(True):
  print('\r', end='')
  print(current_num, end='')
  sleep_seconds = 10
  time.sleep(sleep_seconds)

for thread in thread_list:
    thread.join()

30490
0
1
2
3
4
5
6
7
8
9
10
11
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]

KeyboardInterrupt: ignored

In [ ]:
import os.path

days_start = 1942
days_end = 1969

train_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sales_train_validation.csv")
calendar_data_0 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar.csv")
calendar_data = calendar_data_0[days_start-1:].reset_index(drop=True)
sellprices2_data_0 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices2.csv")
sellprices2_data = sellprices2_data_0[days_start-1:].reset_index(drop=True)
calendar_data2_0 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar2.csv")
calendar_data2 = calendar_data2_0[days_start-1:].reset_index(drop=True)

days_num = days_end - days_start + 1

days_list = ["d_" + str(ii) for ii in range(days_start, days_end + 1)]

cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "wm_yr_wk", "wday", "month", "year", "d",
        "Religious", "National", "Cultural", "Sporting", "snap"] #"price", "price_down", "price_up", "value"は後程追加
train_data2a0 = pd.DataFrame(index=[ij for ij in range(days_num)], columns=cols)

def stap(ST):
    return calendar_data2["snap_" + ST]

for ii in range(30490):
  print("\r"+str(ii), end = " ")
  if (os.path.isfile("/content/drive/My Drive/google_notebook/M5_traindata2/export_data2a/export2a_" + str(ii) + ".csv") == False):  # ファイルがまだ作成されていない
    train_data2a = train_data2a0.copy()
    tra_sir = train_data.iloc[ii]
    for jj in range(days_num):train_data2a.iloc[jj,0:6] = tra_sir[0:6]
    train_data2a.iloc[:,6] = calendar_data.iloc[: ,1] #週
    train_data2a.iloc[:,7:11] = calendar_data.iloc[: ,3:7] #年, 月, 曜日
    train_data2a.iloc[:,11:15] = calendar_data2.iloc[: ,1:5] #イベント情報
    tds_kari = stap(tra_sir[5]) #snap
    train_data2a.iloc[:,15] = tds_kari

    sap_mindata = sellprices2_data[sellprices2_data.item_id == tra_sir[1]]  # sellpricesの中から必要なデータを抽出
    sap_mindata = sap_mindata[sap_mindata.store_id == tra_sir[4]]
    sap_mindata = sap_mindata.drop(["store_id", "item_id"], axis=1)

    train_data2a = pd.merge(train_data2a, sap_mindata, on='wm_yr_wk', how='left') #価格とその変化

    train_data2a.to_csv(
        "/content/drive/My Drive/google_notebook/M5_traindata2/export_data2a/export2a_" + str(ii) + ".csv", index=False)

30489 

In [ ]:
import os
for ii in [2453, 2464, 2470, 2475, 2482, 2493]:
  os.remove("/content/drive/My Drive/google_notebook/M5_traindata2/export_data2a/export2a_" + str(ii) + ".csv")

In [ ]:
main_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/export_data2a/export2a_" + str(0) + ".csv")
main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/export_data2a/export2a_kari.csv", index=False)
with open("/content/drive/My Drive/google_notebook/M5_traindata2/export_data2a/export2a_syukeityu.txt", mode='w') as f:
    f.write(str(1))

In [3]:
size_cols = {"wday": 'int8', "month": 'int8', "year": 'int8',"Religious": 'int8', "National": 'int8',
             "Cultural": 'int8', "Sporting": 'int8', "snap": 'int8'}

main_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/export_data2a/export2a_kari.csv", dtype=size_cols)
with open("/content/drive/My Drive/google_notebook/M5_traindata2/export_data2a/export2a_syukeityu.txt") as f:
    ii = int(f.read())
max_ii = 30489
error_flag = False

while True:
  try:
    df = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/export_data2a/export2a_" + str(ii) + ".csv", dtype=size_cols)
  except:
    print(str(ii)+ "doesn't exist")
    error_flag = True
  else:
    main_data = pd.concat([main_data, df])
    print('\r', end='')
    print(ii, end='')
    if (ii%100 == 0):
      if (error_flag == True): sys.exit(1)
      with open("/content/drive/My Drive/google_notebook/M5_traindata2/export_data2a/export2a_syukeityu.txt", mode='w') as f:
        f.write(str(ii))
      main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/export_data2a/export2a_kari.csv", index=False)
  finally:
    ii += 1
    if (ii > max_ii): break
    
  

main_data.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/export_data2a/export2a_kari.csv", index=False)

30489

In [4]:
import gc

size_cols = {"wday": 'int8', "month": 'int8', "year": 'int8',"Religious": 'int8', "National": 'int8',
             "Cultural": 'int8', "Sporting": 'int8', "snap": 'int8'}

mda = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/export_data2a/export2a_kari.csv", dtype=size_cols)
mda = mda.drop_duplicates()
mda.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/export2a_all.csv", index=False)

del mda
gc.collect()

217

In [5]:
import gc

size_cols = {"wday": 'int8', "month": 'int8', "year": 'int8',"Religious": 'int8', "National": 'int8',
             "Cultural": 'int8', "Sporting": 'int8', "snap": 'int8'}

mda = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/export2a_all.csv", dtype=size_cols)
def d_del(x):
  if ("d_" in x): return x[2:]
  else: return x

mda = mda.drop_duplicates()
mda['d'] = mda['d'].map(d_del)
mda.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/export2a_all.csv", index=False)

del mda
gc.collect()

0

In [6]:
import gc

size_cols = {"wday": 'int8', "month": 'int8', "year": 'int8',"Religious": 'int8', "National": 'int8',
             "Cultural": 'int8', "Sporting": 'int8', "snap": 'int8'}

mda = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/export2a_all.csv", dtype=size_cols)

mda = mda.fillna({'sell_price': '0'})

mda.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/export2a_all.csv", index=False)

del mda
gc.collect()

0

In [3]:
size_cols = {"wday": 'int8', "month": 'int8', "year": 'int8',"Religious": 'int8', "National": 'int8',
             "Cultural": 'int8', "Sporting": 'int8', "snap": 'int8'}

trn = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train2a_all2.csv", dtype=size_cols)
trn2 = trn[trn["month"] >= 5]
trn2 = trn2[trn2["month"] <= 6]

trn2.to_csv("/content/drive/My Drive/google_notebook/M5_traindata2/train2a_56month.csv", index=False)

FileNotFoundError: ignored

In [3]:
import glob
files = glob.glob("/content/drive/My Drive/google_notebook/*")
files

[]